<a href="https://colab.research.google.com/github/KNU-BrainAI-Capstone2022/ITS/blob/main/review_sentiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 32.5 MB/s 
     |████████████████████████████████| 163 kB 59.8 MB/s 
     |████████████████████████████████| 7.6 MB 61.0 MB/s 


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim

from transformers import BertTokenizer, BertForSequenceClassification

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [4]:
path = '/content/drive/MyDrive/fake reviews dataset.csv'

df = pd.read_csv(path)
print(df.shape)
df.head()

(40432, 4)


,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [11]:
df2 = df.loc[(df['label'] == 'OR') & (df['rating'] != 3) ]
print(df2.shape)
df2.head()

(18382, 4)


,category,rating,label,text_
55,Home_and_Kitchen_5,5.0,OR,"These are just perfect, exactly what I was loo..."
56,Home_and_Kitchen_5,5.0,OR,Such a great purchase can't beat it for the price
57,Home_and_Kitchen_5,5.0,OR,What can you say--- cheap and it works as inte...
58,Home_and_Kitchen_5,5.0,OR,"These are so nice, sturdy, like the color choi..."
59,Home_and_Kitchen_5,5.0,OR,It is nice bowl and have had a fast shipping!


In [13]:
df2['division'] = np.select([df2.rating > 3], [1], default=0)
df2[:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,category,rating,label,text_,division
55,Home_and_Kitchen_5,5.0,OR,"These are just perfect, exactly what I was loo...",1
56,Home_and_Kitchen_5,5.0,OR,Such a great purchase can't beat it for the price,1
57,Home_and_Kitchen_5,5.0,OR,What can you say--- cheap and it works as inte...,1
58,Home_and_Kitchen_5,5.0,OR,"These are so nice, sturdy, like the color choi...",1
59,Home_and_Kitchen_5,5.0,OR,It is nice bowl and have had a fast shipping!,1


In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased') # 'bert-base-cased': 영어용, 한국어 및 다른 언어시 변경 필요

# example_text = 'I will watch Memento tonight'
# bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
#                        truncation=True, return_tensors="pt")

# # padding : to pad each sequence to the maximum length that you specify
# # max_length : the maximum length of each sequence.
# #              For our actual dataset we will use 512, which is the maximum length of a sequence allowed for BERT.
# # truncation : if True, then the tokens in each sequence that exceed the maximum length will be truncated.
# # return_tensors : the type of tensors that will be returned. 
# #                  Pytorch: pt. 
# #                  Tensorflow: tf


# print(bert_input['input_ids'])
# # >>>tensor([[  101,   146,  1209,  2824,  2508, 26173,  3568,   102,     0,     0]])
# # tokenizer.decode(bert_input.input_ids[0])
# # >>> [CLS] I will watch Memento tonight [SEP] [PAD] [PAD]
# # BertTokenizer가 알아서 다 해줌
# print(bert_input['token_type_ids'])
# # >>>tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
# print(bert_input['attention_mask'])
# # >>>tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df2):

        self.divisions = [division for division in df2['division']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df2['text_']]
    def classes(self):
        return self.divisions

    def __len__(self):
        return len(self.divisions)

    def get_batch_divisions(self, idx):
        # Fetch a batch of labels
        return np.array(self.divisions[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_divisions(idx)

        return batch_texts, batch_y

df2[:5]

,category,rating,label,text_,division
55,Home_and_Kitchen_5,5.0,OR,"These are just perfect, exactly what I was loo...",1
56,Home_and_Kitchen_5,5.0,OR,Such a great purchase can't beat it for the price,1
57,Home_and_Kitchen_5,5.0,OR,What can you say--- cheap and it works as inte...,1
58,Home_and_Kitchen_5,5.0,OR,"These are so nice, sturdy, like the color choi...",1
59,Home_and_Kitchen_5,5.0,OR,It is nice bowl and have had a fast shipping!,1


In [22]:
np.random.seed(112)
df2_train, df2_val, df2_test = np.split(df2.sample(frac=1, random_state=42), 
                                     [int(.7*len(df2)), int(.9*len(df2))]) # train: 70, validation: 20, test: 10

print(len(df2_train),len(df2_val), len(df2_test))

12867 3676 1839


In [23]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased') # 'bert-base-cased': 영어용, 다른 언어의 데이터면 다른거 쓰기
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # _: embedding vectors of all of the tokens in a sequence
        # pooled_output: embedding vector of [CLS] token, 분류 문제에선 이것만 써도 충분
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [24]:
def saveModel(): 
    path = '/content/drive/MyDrive/NetModel2.pth'
    torch.save(model.state_dict(), path)

In [27]:
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs, cuda_switch):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr= learning_rate)

    if cuda_switch:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            best_accuracy = 0

            for train_input, train_division in tqdm(train_dataloader):

                train_division = train_division.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_division)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_division).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_division in val_dataloader:

                    val_division = val_division.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_division)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_division).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
            accuracy= (total_loss_val / len(val_data))

            if accuracy > best_accuracy: 
               saveModel() 
               best_accuracy = accuracy 

EPOCHS = 8 
model = BertClassifier()
LR = 1e-6
              
train(model, df2_train, df2_val, LR, EPOCHS, use_cuda)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 6434/6434 [08:56<00:00, 11.98it/s]


Epochs: 1 | Train Loss:  0.138                 | Train Accuracy:  0.884                 | Val Loss:  0.104                 | Val Accuracy:  0.909


100%|██████████| 6434/6434 [08:58<00:00, 11.94it/s]


Epochs: 2 | Train Loss:  0.074                 | Train Accuracy:  0.943                 | Val Loss:  0.084                 | Val Accuracy:  0.936


100%|██████████| 6434/6434 [08:58<00:00, 11.94it/s]


...

Epochs: 6 | Train Loss:  0.010                 | Train Accuracy:  0.996                 | Val Loss:  0.137                 | Val Accuracy:  0.943


100%|██████████| 6434/6434 [09:00<00:00, 11.90it/s]


Epochs: 7 | Train Loss:  0.009                 | Train Accuracy:  0.996                 | Val Loss:  0.140                 | Val Accuracy:  0.945


100%|██████████| 6434/6434 [08:59<00:00, 11.94it/s]


Epochs: 8 | Train Loss:  0.007                 | Train Accuracy:  0.997                 | Val Loss:  0.145                 | Val Accuracy:  0.945


In [28]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_division in test_dataloader:

              test_division = test_division.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_division).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df2_test)

Test Accuracy:  0.941


In [29]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()
    path = "/content/drive/MyDrive/NetModel2.pth"
    model.load_state_dict(torch.load(path))
    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df2_test)

Test Accuracy:  0.940
